In [88]:
url = "https://raw.githubusercontent.com/codebasics/py/refs/heads/master/ML/18_PCA/Exercise/heart.csv"

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

In [90]:
heart_df = pd.read_csv(url)
heart_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [91]:
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [92]:
heart_df[heart_df.Cholesterol>(heart_df.Cholesterol.mean()+3*heart_df.Cholesterol.std())]
# records that are outside z score 3

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
76,32,M,ASY,118,529,0,Normal,130,N,0.0,Flat,1
149,54,M,ASY,130,603,1,Normal,125,Y,1.0,Flat,1
616,67,F,NAP,115,564,0,LVH,160,N,1.6,Flat,0


In [93]:
heart_df_1 = heart_df[heart_df.Cholesterol <= (heart_df.Cholesterol.mean()+3*heart_df.Cholesterol.std())]

In [94]:
heart_df_1[heart_df_1.MaxHR>(heart_df_1.MaxHR.mean()+3*heart_df_1.MaxHR.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [95]:
heart_df_1[heart_df_1.Oldpeak>(heart_df_1.Oldpeak.mean()+3*heart_df_1.Oldpeak.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
166,50,M,ASY,140,231,0,ST,140,Y,5.0,Flat,1
702,59,M,TA,178,270,0,LVH,145,N,4.2,Down,0
771,55,M,ASY,140,217,0,Normal,111,Y,5.6,Down,1
791,51,M,ASY,140,298,0,Normal,122,Y,4.2,Flat,1
850,62,F,ASY,160,164,0,LVH,145,N,6.2,Down,1
900,58,M,ASY,114,318,0,ST,140,N,4.4,Down,1


In [96]:
heart_df_2 = heart_df_1[heart_df_1.Oldpeak <= (heart_df_1.Oldpeak.mean()+3*heart_df_1.Oldpeak.std())]

In [97]:
heart_df_1[heart_df_1.RestingBP>(heart_df_1.RestingBP.mean()+3*heart_df_1.RestingBP.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
109,39,M,ATA,190,241,0,Normal,106,N,0.0,Up,0
241,54,M,ASY,200,198,0,Normal,142,Y,2.0,Flat,1
365,64,F,ASY,200,0,0,Normal,140,Y,1.0,Flat,1
399,61,M,NAP,200,0,1,ST,70,N,0.0,Flat,1
592,61,M,ASY,190,287,1,LVH,150,Y,2.0,Down,1
732,56,F,ASY,200,288,1,LVH,133,Y,4.0,Down,1
759,54,M,ATA,192,283,0,LVH,195,N,0.0,Up,1


In [98]:
heart_df_3 = heart_df_2[heart_df_2.RestingBP <= (heart_df_2.RestingBP.mean()+3*heart_df_2.RestingBP.std())]

In [99]:
heart_df_3.ExerciseAngina.describe()

,ExerciseAngina
count,902
unique,2
top,N
freq,539


In [100]:
# map sex, exercise to integers # can use encoding here as well

heart_df_3.loc[:, 'Sex'] = heart_df_3['Sex'].map({"M": 1, "F": 0})
heart_df_3.loc[:, 'ExerciseAngina'] = heart_df_3['ExerciseAngina'].map({"Y": 1, "N": 0})

heart_df_3.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,ATA,140,289,0,Normal,172,0,0.0,Up,0
1,49,0,NAP,160,180,0,Normal,156,0,1.0,Flat,1
2,37,1,ATA,130,283,0,ST,98,0,0.0,Up,0
3,48,0,ASY,138,214,0,Normal,108,1,1.5,Flat,1
4,54,1,NAP,150,195,0,Normal,122,0,0.0,Up,0


In [101]:
heart_df_3 = pd.get_dummies(heart_df_3, drop_first=True, dtype=int)
heart_df_3.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_1,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_1,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0.0,0,1,0,1,0,1,0,0,0,1


In [102]:
ip = heart_df_3.drop("HeartDisease", axis=1)

In [103]:
op = heart_df_3.HeartDisease

In [104]:
# scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
ip_scaled = scaler.fit_transform(ip)

In [105]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1, 5, 10]
        }
    },
    'linear_regression': {
        'model': LinearRegression(),
        'params': {
            'fit_intercept': [True, False]
        }
    }
}

In [106]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(ip_scaled, op)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [107]:
scores_df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
scores_df

,model,best_score,best_params
0,svm,0.833653,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.824764,{'n_estimators': 10}
2,linear_regression,0.473259,{'fit_intercept': True}


In [108]:
# use pca
from sklearn.decomposition import PCA
pca = PCA(0.95)

ip_pca = pca.fit_transform(ip_scaled)

In [109]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(ip_pca, op)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [110]:
scores_df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
scores_df

,model,best_score,best_params
0,svm,0.831436,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.794843,{'n_estimators': 5}
2,linear_regression,0.479310,{'fit_intercept': True}
